# Prereq
First load data

In [1]:
import csv
import numpy as np
from __future__ import print_function
from keras.models import Sequential
from keras import layers
from six.moves import range

Using TensorFlow backend.


In [2]:
INFIL = '/Users/prasad/Desktop/data/male_name_coded/IMN_short.csv'
y_x = []
with open(INFIL) as inF:
    rdr = csv.reader(inF)
    for row in rdr:
        y_x.append((row[0],row[1]))
len(y_x), y_x[:10]        

(44538,
 [('name', 'nme'),
  ('name', 'nm'),
  ('name', 'name'),
  ('barjraj', 'brjraj'),
  ('barjraj', 'brjrj'),
  ('barjraj', 'barjraj'),
  ('ramdin verma', 'rmdin verma'),
  ('ramdin verma', 'rmdn vrm'),
  ('ramdin verma', 'ramdin verma'),
  ('sharat chandran', 'shrat chandran')])

In [3]:
max_x_len, max_y_len = -1,-1
for y,x in y_x:
    if len(y) > max_y_len:
        max_y_len = len(y)
    if len(x) > max_x_len:
        max_x_len = len(x)
(max_x_len, max_y_len)

(37, 37)

In [4]:
all_chars = set()

def register(s,a_set):
    for c in s:
        a_set.add(c)

for y,x in y_x:
    register(y,all_chars)
    register(y,all_chars)


In [5]:
len(all_chars)

84

In [6]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [25]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [8]:
TRAINING_SIZE = len(y_x)
reverse = True

In [9]:
ctable = CharacterTable(all_chars)

In [10]:
questions = []
expected = []
seen = set()

In [11]:
x_l = []
y_l = []

for y,x in y_x:
    if (y,x) in seen:
        continue
    seen.add((y,x))
    pad_x = x + ' ' * (max_x_len - len(x))
    pad_y = y + ' ' * (max_y_len - len(y))
    if reverse:
        pad_x = pad_x[::-1]
    x_l.append(pad_x)
    y_l.append(pad_y)
    

In [12]:
len(x_l),len(y_l)

(23897, 23897)

In [13]:
x_arr = np.zeros((len(x_l), max_x_len, len(all_chars)), dtype=np.bool)
y_arr = np.zeros((len(y_l), max_y_len, len(all_chars)), dtype=np.bool)

In [14]:
for i, sentence in enumerate(x_l):
    x_arr[i] = ctable.encode(sentence, max_x_len)
for i, sentence in enumerate(y_l):
    y_arr[i] = ctable.encode(sentence, max_y_len)

# Shuffle the data

In [15]:
indices = np.arange(len(y_arr))
np.random.shuffle(indices)
x_arr = x_arr[indices]
y_arr = y_arr[indices]

# 10 % for validation

In [16]:
split_at = len(x_arr) - len(x_arr) // 10
(x_train, x_val) = x_arr[:split_at], x_arr[split_at:]
(y_train, y_val) = y_arr[:split_at], y_arr[split_at:]

'Training Data', x_train.shape, y_train.shape,'  Validation Data',x_val.shape,y_val.shape

('Training Data',
 (21508, 37, 84),
 (21508, 37, 84),
 '  Validation Data',
 (2389, 37, 84),
 (2389, 37, 84))

In [17]:
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

# Build Model

In [18]:
model = Sequential()

"Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
Note: In a situation where your input sequences have a variable length,
use 
```
input_shape=(None, num_feature).
```

In [19]:
model.add(RNN(HIDDEN_SIZE, input_shape=(max_x_len, len(all_chars))))

As the decoder RNN's input, repeatedly provide with the last hidden state of
RNN for each time step. Repeat `max_y_len`  times as that's the maximum
length of output

In [20]:
model.add(layers.RepeatVector(max_y_len))

The decoder RNN could be multiple layers stacked or a single layer.

By setting return_sequences to True, return not only the last output but
all the outputs so far in the form of (num_samples, timesteps,
output_dim). This is necessary as TimeDistributed in the below expects
the first dimension to be the timesteps.

In [21]:
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

Apply a dense layer to the every temporal slice of an input. For each of step
of the output sequence, decide which character should be chosen.

In [22]:
model.add(layers.TimeDistributed(layers.Dense(len(all_chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               109056    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 37, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 37, 128)           131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 37, 84)            10836     
_________________________________________________________________
activation_1 (Activation)    (None, 37, 84)            0         
Total params: 251,476
Trainable params: 251,476
Non-trainable params: 0
_________________________________________________________________


Train the model each generation and show predictions against the validation
dataset.


In [ ]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if reverse else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 21508 samples, validate on 2389 samples
Epoch 1/1
21508/21508 [==============================] - 23s 1ms/step - loss: 0.9360 - acc: 0.7393 - val_loss: 0.8961 - val_acc: 0.7395
Q rpan singh                            T rupan singh                           ☒ aaaaaaaaa                            
Q kash urf deepu                        T akash urf deepu                       ☒ aaaaaaaaaaa                          
Q pnt chwl                              T puneet chawla                         ☒ aaaaaaa                              
Q rbl khrn                              T rubal khurana                         ☒ aaaaaaaa                             
Q xanwlal@golu                          T santlal@golu                          ☒ aaaaaaaaaaa                          
Q lxman dass                            T laxman dass                           ☒ aaaaaaaaa                            
Q krnkr dhra                   

21508/21508 [==============================] - 24s 1ms/step - loss: 0.7699 - acc: 0.7697 - val_loss: 0.7695 - val_acc: 0.7669
Q grish chand                           T girish chand                          ☒ sahhsssssshhh                        
Q bhln kmr                              T bhulan kumar                          ☒ sanal   mmma                         
Q tpn kmr                               T tapan kumar                           ☒ mane  kkmaa                          
Q shb bbs                               T shabi abbas                           ☒ shhhb  aaaaa                         
Q hmnsh shrm                            T himanshu sharma                       ☒ shhhhh   aaaaa                       
Q tfk                                   T taufik                                ☒ aaaa                                 
Q gngadhar                              T gangadhar                             ☒ sanessaaa                            
Q pawan dev mehwo                 

21508/21508 [==============================] - 23s 1ms/step - loss: 0.5554 - acc: 0.8189 - val_loss: 0.5554 - val_acc: 0.8189
Q jaxim ahemad                          T jasim ahemad                          ☒ jaii   mmmd                          
Q bdl khll                              T abdul khalil                          ☒ bddll llllll                         
Q fzn                                   T faizan                                ☒ aaziz                                
Q deewan                                T deewan                                ☒ deeean                               
Q skhrm                                 T sukhram                               ☒ sakhammm                             
Q nrsh chhn                             T naresh chauhan                        ☒ narishhhhhan                         
Q anoop xingh                           T anoop singh                           ☒ nono  sigg                           
Q hri mohan gurjar                

21508/21508 [==============================] - 24s 1ms/step - loss: 0.4259 - acc: 0.8583 - val_loss: 0.5255 - val_acc: 0.8224
Q khmchnd                               T khamchand                             ☒ khakcaadd                            
Q sndp gal                              T sandeep goyal                         ☒ sandep aal                           
Q chndan adiwasi                        T chandan adiwasi                       ☒ chandn aaiiiai                       
Q lela dhar narang                      T leela dhar narang                     ☒ lalaa aaaaaaaan                      
Q dinexh xingh rana                     T dinesh singh rana                     ☒ diesshiinrrraai                      
Q mngl bnd                              T mangal bind                           ☒ mani  bbadd                          
Q shr  rupa ram                         T shri  rupa ram                        ☒ shrrrrrrrraam                        
Q vikaxh                          

21508/21508 [==============================] - 25s 1ms/step - loss: 0.3459 - acc: 0.8854 - val_loss: 0.3643 - val_acc: 0.8806
Q vrndr sngh                            T virender singh                        ☒ vivendrrsinngh                       
Q ajay xharma                           T ajay sharma                           ☒ ajay sharaa                          
Q shjd                                  T sahajad                               ☒ saaaad                               
Q kaxhixh xharma                        T kashish sharma                        ☒ kassssh sharma                       
Q sndp pwr                              T sandeep pawar                         ☒ sandeepppawaa                        
Q xhree nanak                           T shree nanak                           ☒ shreen nnna                          
Q tp mhd                                T tepu mohd                             ☒ ttpp ahha                            
Q dhrmndr sngh                    

21508/21508 [==============================] - 23s 1ms/step - loss: 0.2867 - acc: 0.9064 - val_loss: 0.3115 - val_acc: 0.8984
Q ml tppo                               T emilia toppo                          ☒ ma   toppp                           
Q ankiw verma                           T ankit verma                           ☒ ankit veema                          
Q rohiw yadav                           T rohit yadav                           ☑ rohit yadav                          
Q lck                                   T lucky                                 ☒ lacku                                
Q jwed ahmad                            T jawed ahmad                           ☒ jeee  hamd                           
Q nthn mndl                             T nathani mandal                        ☒ nathnn manda                         
Q rtknt mgh                             T ratikant maghi                        ☒ rakkant  ohh                         
Q man kumar                       

21508/21508 [==============================] - 23s 1ms/step - loss: 0.2498 - acc: 0.9196 - val_loss: 0.2878 - val_acc: 0.9085
Q harpreew xingh bedi                   T harpreet singh bedi                   ☒ harreeer singh iidi                  
Q klwinder singh                        T kulwinder singh                       ☒ kallinder singh                      
Q nveen rawat                           T naveen rawat                          ☒ neveen rawat                         
Q kuxh gupwa                            T kush gupta                            ☒ kush ggpta                           
Q p dnesh raju                          T p dinesh raju                         ☒ pupdnndshrraj                        
Q srfrj                                 T sarfaraj                              ☒ sarfaaj                              
Q snl                                   T sunil                                 ☒ sanil                                
Q mnsh lmb                        